In [1]:
import pandas as pd
import numpy as np
import csv
import pickle

In [2]:
sample  = pickle.load( open( "twitter_swisscom_proc/xaa_proc.p", "rb" ) )
sample.head()

,id,userId,text,longitude,latitude,inReplyTo,placeLatitude,placeLongitude,followersCount,friendsCount,statusesCount,userLocation,year,month,day,week_number,hour,city,state,country
551,10231423626,6257282.0,"The new apartment is nice, but there is no Wif...",7.58531,47.5455,NaN,47.5367,7.57849,14249,9260.0,19585.0,"Potsdam, Germany",2010.0,3.0,9.0,10.0,18.0,Binningen,Basel-Landschaft,ch
609,10292646240,15602037.0,Is that wet yet solid stuff on my screen suppo...,8.52725,47.3876,NaN,47.3791,8.50021,177,136.0,5167.0,"Zürich, Switzerland",2010.0,3.0,10.0,10.0,22.0,Zürich,Zürich,ch
611,10309829732,625553.0,I'm at DCTI - David Dufour in Geneva http://go...,6.13183,46.2006,NaN,46.1996,6.13011,471,82.0,3363.0,"Geneva, Switzerland",2010.0,3.0,11.0,10.0,5.0,Genève,Genève,ch
612,10310391132,17341045.0,God morgon! :-),7.44235,46.8957,NaN,46.9214,7.38855,586,508.0,9016.0,"Bern, Switzerland",2010.0,3.0,11.0,10.0,6.0,Köniz,Bern - Berne,ch
618,10311568050,634553.0,"At this very minute, the sun is pink.",6.19900,46.2043,NaN,46.1938,6.15415,2230,387.0,10605.0,"Geneva, Switzerland",2010.0,3.0,11.0,10.0,7.0,Genève,Genève,ch


In [3]:
sample.isnull().sum()

id                    0
userId                0
text                  0
longitude          4514
latitude           4514
inReplyTo         17246
placeLatitude         0
placeLongitude        0
followersCount        0
friendsCount          0
statusesCount         0
userLocation        415
year                  0
month                 0
day                   0
week_number           0
hour                  0
city                 41
state                 3
country               0
dtype: int64

In [4]:
sample.shape

(22611, 20)

A few cities and states is not found, so we can drop it.

In [5]:
user=sample.dropna(subset=['state'])
user.isnull().sum()

id                    0
userId                0
text                  0
longitude          4514
latitude           4514
inReplyTo         17243
placeLatitude         0
placeLongitude        0
followersCount        0
friendsCount          0
statusesCount         0
userLocation        415
year                  0
month                 0
day                   0
week_number           0
hour                  0
city                 41
state                 0
country               0
dtype: int64

# Cleaning unusual trajectories

Revocer time for calculating valid user in reasonable moving speed.

In [6]:
import time
import datetime

time=user[['year','month','day','hour']]
temp=pd.to_datetime(time)
temp.head()
user['time']=temp
user.head()

D:\anaconda\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,userId,text,longitude,latitude,inReplyTo,placeLatitude,placeLongitude,followersCount,friendsCount,...,userLocation,year,month,day,week_number,hour,city,state,country,time
551,10231423626,6257282.0,"The new apartment is nice, but there is no Wif...",7.58531,47.5455,NaN,47.5367,7.57849,14249,9260.0,...,"Potsdam, Germany",2010.0,3.0,9.0,10.0,18.0,Binningen,Basel-Landschaft,ch,2010-03-09 18:00:00
609,10292646240,15602037.0,Is that wet yet solid stuff on my screen suppo...,8.52725,47.3876,NaN,47.3791,8.50021,177,136.0,...,"Zürich, Switzerland",2010.0,3.0,10.0,10.0,22.0,Zürich,Zürich,ch,2010-03-10 22:00:00
611,10309829732,625553.0,I'm at DCTI - David Dufour in Geneva http://go...,6.13183,46.2006,NaN,46.1996,6.13011,471,82.0,...,"Geneva, Switzerland",2010.0,3.0,11.0,10.0,5.0,Genève,Genève,ch,2010-03-11 05:00:00
612,10310391132,17341045.0,God morgon! :-),7.44235,46.8957,NaN,46.9214,7.38855,586,508.0,...,"Bern, Switzerland",2010.0,3.0,11.0,10.0,6.0,Köniz,Bern - Berne,ch,2010-03-11 06:00:00
618,10311568050,634553.0,"At this very minute, the sun is pink.",6.19900,46.2043,NaN,46.1938,6.15415,2230,387.0,...,"Geneva, Switzerland",2010.0,3.0,11.0,10.0,7.0,Genève,Genève,ch,2010-03-11 07:00:00


In [7]:
# velticity calculation, miles per hour
from geopy.distance import vincenty 

def speed(loc1, loc2, t1, t2):
    d = vincenty(loc1,loc2).miles
    dt = t2-t1
    dh = dt.seconds/3600.0 #to_hours
    if dh :
        return d/dh 
    else: #within one hour, assume it travels within half an hour
        return d/0.5

In [10]:
#ref: Exploring Multi-Scale Spatiotemporal Twitter User Mobility Patterns with a Visual-Analytics Approach
#Space-Time Twitter User Trajectories
#To remove non-human users based on unusual relocating speed: e.g. speed (loc_i − loc_i−1) > 240 m/s

#Travelling in Switzerland mainly takes train(despite some ones take flight)
#Data are in time-order, so the location is continuously changed
#Since we clean the data before, time is accurate to hour, so we set a loose thedshold like 200 mils/h 


#remove non-human user
def human_user(c_tuple, speed_thed=200):
    lon,lan,t = tuple(zip(*c_tuple))
    point = list(zip(lan,lon))
    normal=[True]*(len(c_tuple))
    
    if len(c_tuple) == 1:
        normal[0] = False # remove only one tweet
    if len(c_tuple) == 2 :
        v = speed(point[0],point[1],t[0],t[1])
        if v > speed_thed:
            normal = [False]*(len(c_tuple))
    else:
        loc1 = point[0:-2]
        loc2 = point[1:-1]
        loc3 = point[2::]
        #Using geopy's vincenty to find distances in pandas data frame
        for i in range(len(loc1)):
            v1_2=speed(loc2[i],loc1[i],t[i],t[i+1])
            v2_3=speed(loc2[i],loc3[i],t[i+1],t[i+2])
            v1_3=speed(loc1[i],loc3[i],t[i],t[i+2])
        
            # Check if loc1 is a normal location with reasonable speed
            # If there is something wrong with loc1->loc2, we check loc2->loc3 & loc1->loc3 to ensure if loc2/3 is wrong
        
            if v1_2 > speed_thed: # loc1->loc2 innormal
                if v2_3 > speed_thed: # loc2->loc1/3 innormal -> loc2 innormal
                    normal[i+1]=False 
                    if v1_3 > speed_thed: # loc1->loc2/3 innormal -> loc1 innormal
                        normal[i]=False 
                        if i == len(loc1)-1: # last index with all wrong
                            normal[i+2]=False
                else: # loc1->loc2 innormal, loc2->loc1/3 normal -> loc2 normal, loc1 innormal
                    normal[i]=False
            else: # loc1->loc2 normal
                if v2_3 > speed_thed:
                    if v1_3 > speed_thed: # loc3->loc1/2 innormal -> loc3 innormal
                        normal[i+2]=False
                    else:
                        if (i == len(loc1)-1): # last index, just loc2->loc3 innormal -> loc2 innormal
                            normal[i+1]=False
        
    return normal

In [11]:
#twitter_user=twitter_data.groupby(twitter_data['userId'])
#.order(ascending=True)
user_trace=['userId','year','month','week_number','day']
user['temp'] = tuple(zip(user['placeLatitude'],user['placeLongitude'],user['time']))
normal = user.groupby(by=user_trace)['temp'].transform(lambda x: human_user(x))

D:\anaconda\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
normal_user = user[normal].reset_index(drop=True)
normal_user.shape

(15690, 22)

# Store trace

Next,we hope to define daily trace of every user.

In [13]:
# Since tweets are in time order, we can just store daily trace by apprending locations(canton)

def daily_trace(r_location):
    start=r_location.values[0:-1]
    end=r_location.values[1::]
    index=(start!=end)
    trace=np.nan
    
    if index.any(): # Remove inner-canton trace 
        trace='->'.join(start[index])
        trace='->'.join([trace,end[index][-1]])
        
    return trace    
    

In [14]:
user_dtrace = normal_user.groupby(by=user_trace)['state'].apply(lambda x: daily_trace(x))


In [15]:
user_dtrace=user_dtrace.dropna()
user_dtrace.head()

userId   year    month  week_number  day 
12187.0  2011.0  1.0    4.0          25.0               Basel-Stadt->Aargau
18123.0  2010.0  10.0   43.0         28.0                    Zürich->Aargau
36863.0  2011.0  1.0    4.0          26.0    Basel-Landschaft->Bern - Berne
53453.0  2010.0  12.0   51.0         20.0                   Vaud->Neuchâtel
                                     21.0           Neuchâtel->Bern - Berne
Name: state, dtype: object

In [16]:
with open ( "state_mov.pkl", "wb" ) as output:
    pickle.dump(temp, output, pickle.HIGHEST_PROTOCOL)